# RNN

*Recurent Neural Network*

再帰型ニューラルネットワーク。  
再帰的な構造を持つニューラルネットワークで、可変長の時系列データを扱うことが得意。

<u>2章: 深層学習を用いた言語モデル</u>では、深層学習を活用して、ある単語から次の単語を予測するモデルを作成した。しかし、これでは文脈を考慮できない。具体的には、2つ以上前の単語を考慮した予測が出来ない。  
RNNを用いることで、独立していた時間ごとの演算が繋がりを持つようになり、文脈を考慮した予測が可能になる。

In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F


---

## RNNの構造

RNNはある時間$t$の入力$x_t$に対して以下のような演算を行い、出力値$h_t$を決定する。

$$
h_t = \mathrm{tanh}(W_x x_t + b_x + W_h h_{t-1} + b_h)
$$

演算の内部で前の時間の出力値$h_{t-1}$を参照していることが分かる。  
演算内容はシンプルに捉えることができて、入力$x_t$と前の時間の出力$h_{t-1}$をそれぞれ線形変換し、それらの和を活性化関数$\mathrm{tanh}$に通しているだけ。線形変換に必要な重みとバイアスが2つずつあるので、パラメータは合計4つ。

実装してみるとこんな感じ。

In [2]:
class RNN(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.fc_input = nn.Linear(input_size, output_size)
        self.fc_output = nn.Linear(output_size, output_size)

    def forward(self, x, h):
        z = self.fc_input(x) + self.fc_output(h)
        return F.tanh(z)

適当なパラメータで時間ごとの入力$x_t$と初めの出力$h_0$を定義し、与える。$h_0$は0ベクトルで良い。

In [3]:
batch_size, input_size, output_size = 2, 3, 4
x1 = torch.randn(batch_size, input_size)
x2 = torch.randn(batch_size, input_size)
h0 = torch.zeros(batch_size, output_size)

rnn = RNN(input_size, output_size)
h1 = rnn(x1, h0)
h1

tensor([[-0.2629,  0.8844, -0.9384, -0.0628],
        [-0.1996, -0.7433,  0.4514,  0.3717]], grad_fn=<TanhBackward0>)

この$h$を用いて次の時間の演算を行う。

In [4]:
h2 = rnn(x2, h1)
h2

tensor([[ 0.2056,  0.0176,  0.0549, -0.2210],
        [ 0.4487, -0.9638,  0.7321,  0.1522]], grad_fn=<TanhBackward0>)

これがRNNの処理の流れである。これを繰り返すことで、時系列的な情報を保持しながら演算を行うことが出来る。

また、基本的には、モデルでの時間ごとの出力にRNNの出力$h_t$をそのまま用いることはない。$h_t$を更に幾つかの層に通して、最終的な出力を決定する。RNNが一つの層になっている感じ。  
$h_t$が直接観測されることはないため、潜在変数や隠れ状態と表現される。変数名に$y$ではなく$h$を使っている理由がそこにある（隠れる=hide）。


---

## PyTorchでの実装

PyTorchにはRNNを実装するためのクラスが用意されている。軽く紹介する。

### `torch.nn.RNNCell`

RNNの一つの時間の演算を行うクラス。↑で実装したものと同じ。  
[RNNCell — PyTorch 2.0 documentation](https://pytorch.org/docs/stable/generated/torch.nn.RNNCell.html)

In [5]:
batch_size, input_size, hidden_size = 2, 3, 4

rnn = nn.RNNCell(input_size, hidden_size)
for params in rnn.parameters():
    print(params, '\n')

Parameter containing:
tensor([[ 0.2661,  0.0389,  0.4985],
        [-0.1947, -0.2701, -0.4863],
        [-0.3076,  0.2085,  0.0889],
        [-0.0135, -0.2515, -0.4663]], requires_grad=True) 

Parameter containing:
tensor([[ 0.0374,  0.3906,  0.3850, -0.2784],
        [ 0.4597,  0.2758, -0.3663,  0.4483],
        [ 0.0875,  0.1915,  0.1057, -0.3124],
        [ 0.1724, -0.2271,  0.0879,  0.1380]], requires_grad=True) 

Parameter containing:
tensor([-0.1643,  0.2985,  0.3658, -0.0438], requires_grad=True) 

Parameter containing:
tensor([ 0.0204,  0.0337, -0.2413, -0.1588], requires_grad=True) 



パラメータが4つ。

In [6]:
x1 = torch.randn(batch_size, input_size)
rnn(x1)

tensor([[-0.9649,  0.9699,  0.4011,  0.8166],
        [-0.5998,  0.8435, -0.3830,  0.6143]], grad_fn=<TanhBackward0>)

$h$を入力しなかった場合は勝手に0ベクトルになる。

### `torch.nn.RNN`

全ての時間の演算をまとめて行うクラス。  
[RNN — PyTorch 2.0 documentation](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html)



In [7]:
batch_size, input_size, hidden_size, seq_length = 2, 3, 4, 5
rnn = nn.RNN(input_size, hidden_size, batch_first=True)

x = torch.randn(batch_size, seq_length, input_size)
y, h = rnn(x)
y.shape

torch.Size([2, 5, 4])

バッチごとに全ての時間の隠れ状態が出力される。  
また`h`には最後の時間の隠れ状態が出力される。

In [8]:
y[:, -1] == h

tensor([[[True, True, True, True],
         [True, True, True, True]]])

In [9]:
h.shape

torch.Size([1, 2, 4])

初めに次元が1つ追加されるのは仕様。